## Postprocessing - creating the summary report

In [ ]:
# Import our dependencies
from google.colab import drive
from google.colab import files
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
uploaded = files.upload()

Saving predictions.csv to predictions.csv


In [ ]:
df = pd.read_csv('/content/predictions.csv')

In [ ]:
df['Avg_Days_per_DrugTest'] = df['Avg_Days_per_DrugTest'].\
astype(str).str.replace(',', '').astype(float).replace(0, 1)

In [ ]:
df['Predicted'] = df['Predicted'].astype(bool)

In [ ]:
# prompt: create failed_df from df where df['Predicted'] does not equal df['Actual']
failed_df = df[df['Predicted'] != df['Actual']]

In [ ]:
def categorize_data(data, column, bins, labels):
    data = data.copy()
    data.loc[:, column] = pd.cut(data.loc[:, column].astype(float), bins=bins, labels=labels)
    return data

# Define the parameters for each column
parameters = [
    ('Avg_Days_per_DrugTest', [0, 1, 7, 30, 91, 181, 365, 954], ['daily', 'weekly', 'monthly', 'quarterly', 'bi-annually', 'annually', 'over a year']),
    ('Percent_Days_Employed', [-1, 0, .5, .9999999, 1], ['never', 'some', 'most', 'all']),
    ('Jobs_Per_Year', [-1, 0, .5, .9999999, 1, 5], ['none', 'under employed', 'almost one', 'one', 'multiple']),
    ('DrugTests_THC_Positive', [-1, 0, .9999999, 1], ['never', 'less than one', 'one']),
    ('DrugTests_Cocaine_Positive', [-1, 0, .9999999, 1], ['never', 'less than one', 'one']),
    ('DrugTests_Meth_Positive', [-1, 0, .9999999, 1], ['never', 'less than one', 'one']),
    ('DrugTests_Other_Positive', [-1, 0, .9999999, 1], ['never', 'less than one', 'one'])
]

# Apply the function to each column
for column, bins, labels in parameters:
    failed_df = categorize_data(failed_df, column, bins, labels)

In [ ]:
# Initialize an empty list to store the DataFrames
df_list = []

for column in failed_df.columns:
    unique_values = failed_df[column].unique()
    for value in unique_values:
        true_count = len(failed_df[(failed_df[column] == value) & (failed_df['Predicted'] == True)])
        false_count = len(failed_df[(failed_df[column] == value) & (failed_df['Predicted'] == False)])
        total_count = true_count + false_count

        # Create a dictionary with the calculated counts and percentages
        counts_dict = {
            'column_name': column,
            'column_value': value,
            'predicted_to_recidivise': true_count,
            'predicted_to_recidivise_percent': round((true_count / total_count * 100), 2),
            'predicted_to_succeed': false_count,
            'predicted_to_succeed_percent': round((false_count / total_count * 100),2)
        }

        # Convert the dictionary to a DataFrame and add it to the list
        df_list.append(pd.DataFrame([counts_dict]))

# Concatenate all the DataFrames in the list
summary_df = pd.concat(df_list, ignore_index=True)

# Now 'summary_df' contains the counts and percentages organized in a tabular format
summary_df

,column_name,column_value,predicted_to_recidivise,predicted_to_recidivise_percent,predicted_to_succeed,predicted_to_succeed_percent
0,Gender,M,226,54.99,185,45.01
1,Gender,F,35,54.69,29,45.31
2,Race,WHITE,118,53.88,101,46.12
3,Race,BLACK,143,55.86,113,44.14
4,Age_at_Release,3337,39,52.00,36,48.00
...,...,...,...,...,...,...
216,Training_Sample,1,186,56.02,146,43.98
217,Predicted,True,261,100.00,0,0.00
218,Predicted,False,0,0.00,214,100.00
219,Actual,False,261,100.00,0,0.00


In [ ]:
filename = f"summary.csv"
summary_df.to_csv(filename, index=False)
files.download(filename)
print(f"{filename} has been exported to your Downloads folder.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

summary.csv has been exported to your Downloads folder.
